In [1]:
#### import sys
import numpy as np
import cv2
from time import sleep
from datetime import datetime
import os
import numpy as np
from matplotlib import pyplot as plt
from pandas import read_csv, read_excel, DataFrame
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.linear_model import  LinearRegression
from sklearn.pipeline import make_pipeline
import pickle
from sklearn.preprocessing import PolynomialFeatures
import logging
import hvplot.pandas
logging.basicConfig(level=logging.INFO)
p1 = os.path.join('../../','raw','CookIRCamET','Images','NsarIRCal')
p3 = os.path.join('../../','raw','CookIRCamET','Images','CprlIRCal')
p5 = os.path.join('../../','raw','CookIRCamET','Images','NsarIRCalFlir2')
p7 = os.path.join('../../','raw','CookIRCamET','Images','NsarIRCalFlir3')
p9 = os.path.join('../../','raw','CookIRCamET','Images','NsarIRCalFlirV2')
p11 = os.path.join('../../','raw','CookIRCamET','Images','NsarIRCalFlirField')
p13 = os.path.join('../../','raw','CookIRCamET','Images','NsarIRCalFlirV3')
p15 = os.path.join('/90daydata/nsaru','raw','CookIRCamET','Images','CprlIRCalV3')
p17 = os.path.join('/90daydata/nsaru','work','CookIRCamET','Working')

In [ ]:
files = []
temps = []
pixes = []
for f in os.listdir(p15):
    if 'ir' in f:
        files.append(f)
        temps.append(float(f.split('_')[0]+'.'+f.split('_')[1])+273.15)
        if temps[-1]-273.15<25:
            tempwin = cv2.imread(os.path.join(p15,f),cv2.IMREAD_UNCHANGED)[90-10:90+10,80-10:80+10]
        elif temps[-1]-273.15>25:
            tempwin = cv2.imread(os.path.join(p15,f),cv2.IMREAD_UNCHANGED)[50-10:50+10,101-10:101+10]
        pixes.append(np.mean(tempwin))
        
raw_vals = np.array(pixes)
raw_temps = np.array(temps)
order = np.argsort(raw_vals)

In [72]:
raw_vals = raw_vals[order]
raw_temps = raw_temps[order]

In [73]:
raw_vals = raw_vals[raw_temps<301]
raw_temps = raw_temps[raw_temps<301]

In [74]:
fit1 = LinearRegression().fit(raw_vals[raw_temps<288].reshape(-1,1),raw_temps[raw_temps<288].reshape(-1,1))
fit2 = LinearRegression().fit(raw_vals[(raw_temps>=288) & (raw_temps<=296)].reshape(-1,1),raw_temps[(raw_temps>=288) & (raw_temps<=296)].reshape(-1,1))
fit3 = LinearRegression().fit(raw_vals[raw_temps>296].reshape(-1,1),raw_temps[raw_temps>296].reshape(-1,1))

In [75]:
fit1.coef_[[0]],fit2.coef_,fit3.coef_,fit1.intercept_,fit2.intercept_,fit3.intercept_

(array([[0.01091766]]),
 array([[0.01925118]]),
 array([[0.00880147]]),
 array([-46.75818195]),
 array([-302.40465789]),
 array([23.84508168]))

In [76]:
I0 = (fit1.intercept_[0]-fit2.intercept_[0])/(fit2.coef_[0][0]-fit1.coef_[0][0])

In [77]:
I1 = (fit2.intercept_[0]-fit3.intercept_[0])/(fit3.coef_[0][0]-fit2.coef_[0][0])

In [78]:
df = DataFrame({'raw':raw_vals,'temp':raw_temps,
                'fit1_temp':fit1.predict(raw_vals.reshape(-1,1)).reshape(-1),
                'fit2_temp':fit2.predict(raw_vals.reshape(-1,1)).reshape(-1),
                'fit3_temp':fit3.predict(raw_vals.reshape(-1,1)).reshape(-1)})

In [79]:
fit_temp = np.piecewise(raw_vals, [raw_vals < I0, (raw_vals>=I0) & (raw_vals<I1), raw_vals>=I1], [lambda x: fit1.intercept_[0]+fit1.coef_[0][0]*x, lambda x: fit2.intercept_[0]+fit2.coef_[0][0]*x, lambda x: fit3.intercept_[0]+fit3.coef_[0][0]*x])

In [80]:
df['fit_temp']=fit_temp

In [81]:
with open(os.path.join(p17,'calibration_cprl_pwlf.pk.sav'), 'wb') as f:  # Python 3: open(..., 'wb'
    pickle.dump([fit1, fit2, fit3, I0, I1], f)

In [85]:
df.hvplot.scatter(x='raw',y=['temp','fit_temp'])

:NdOverlay   [Variable]
   :Scatter   [raw]   (value)

In [84]:
filename = os.path.join(p17,'calibration_cprl_lf_v3.pk.sav')
import pickle
with open(filename, 'wb') as f:
    pickle.dump(fit2, f)

In [ ]:
import logging
import traceback
logging.basicConfig(level=logging.INFO)
from utils_segmentation import *

#cal_nsar_v0 = pickle.load(open(os.path.join(p3,'calibration_nsar1.pk.sav'), 'rb'))#20220705
#cal_nsar_v1 = pickle.load(open(os.path.join(p3,'calibration_nsar2.pk.sav'), 'rb'))#20221122
#cal_nsar_v2 = pickle.load(open(os.path.join(p3,'calibration_nsar3.pk.sav'), 'rb'))#20230315
[model0,model1,model2,I0,I1] = pickle.load(open(os.path.join(p3,'calibration_nsar_pwlf.pk.sav'), 'rb'))#V3
#[model0_v3,model1_v3,model2_v3,I0_v3,I1_v3] = pickle.load(open(os.path.join(p3,'calibration_nsar_pwlf_v3.pk.sav'), 'rb'))#V3
cal_nsar_v3 = pickle.load(open(os.path.join(p3,'calibration_nsar_lf_v3.pk.sav'), 'rb'))#V3
cal_cprl_v3 = pickle.load(open(os.path.join(p3,'calibration_cprl_lf_v3.pk.sav'), 'rb'))#V3

cal_cprl = pickle.load(open(os.path.join(p3,'calibration_cprl1.pk.sav'), 'rb'))#20230305


print('%3.4fx%3.4f'%(cal_cprl.coef_[0][0],cal_cprl.intercept_))
print('%3.4fx%3.4f'%(cal_cprl_v3.coef_[0][0],cal_cprl_v3.intercept_))
#print('%3.4fx%3.4f'%(cal_nsar_v2.coef_[0][0],cal_nsar_v2.intercept_))
print('%3.4fx%3.4f'%(cal_nsar_v3.coef_[0][0],cal_nsar_v3.intercept_))
print('%3.4fx+%3.4f x<%3.4f'%(model0.coef_[0][0],model0.intercept_,I0))
print('%3.5fx+%3.4f %3.5f<=x<%3.4f'%(model1.coef_[0][0],model1.intercept_,I0,I1))
print('%3.4fx+%3.4f x>=%3.4f'%(model2.coef_[0][0],model2.intercept_,I1))

In [ ]:
stop_dates_cprl_v3 = pd.read_excel(os.path.join(p11,'StartStopDates.xlsx'),dtype={'Start':str,'Stop':str}).Stop.str.strip("'").values.tolist()
